In [23]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout


In [ ]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=10&mid1=1'

In [86]:
async def get_html(url, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                await page.mouse.wheel(15000,15000)
                time.sleep(2)
                html = await page.innerHTML('body')
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [87]:
html = get_html(url)

In [103]:
leagues0 = []

In [104]:

for i in range (1,100):
    try:
        df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/{i}/settings') 
        dic ={}
        dic['id'] = i
        dic['format'] = df[1]
        leagues0.append(dic)
    except:
        pass
    
    time.sleep(5)


In [55]:
response = requests.get(url)

In [56]:
soup = BeautifulSoup(response.text,'html.parser')

In [75]:
html = await get_html(url)

Fantasy Football | Yahoo! Sports


In [76]:
html

'<!DOCTYPE html><html id="Stencil" class="template-html5 Sticky-off Desktop JsEnabled yui3-js-enabled stencil-js stencil-no-touch stencil-csstransforms3d stencil-csstransitions stencil-svg stencil-fullscreen stencil-plugin-selectbox stencil-plugin-imageloader stencil-plugin-source stencil-plugin-tooltip has-scrolled" xmlns:fb="https://www.facebook.com/2008/fbml" lang="en-US"><div id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important" class=""></div><head>\n    <meta http-equiv="X-UA-Compatible" content="IE=EDGE">\n    <link rel="canonical" href="https://football.fantasysports.yahoo.com/f1/14104/matchup?week=10&amp;mid1=1">\n    <link rel="manifest" href="/manifest.json">\n    <link rel="dns-prefetch" href="https://s.yimg.com">\n<link rel="dns-prefetch" href="//csc.beap.bc.yahoo.com">\n<link rel="dns-prefetch" href="//geo.query.yahoo.com">\n<link rel="dns-prefetch" href="//y.analytics.yahoo.com">\n<link rel="dns-prefetch" href="//b.scorecardresearch.com